# 3D Molecule Equivalence Testing
Author: Connor Davel

Date: 1/9/2021
## Summary
This notebook proposes a few useful methods of cross testing the RDKit and OpenEye toolkits currently integrated in openff. A "bottom-up" approach was used, testing 371 molecules from the MiniDrugBank.sdf. Overall, about 90% of molecules passed all tests for "reasonable" equivalence. The causes for failure could be mostly narrowed down to ambiguities in implicit protons, formal charges, and stereochemistry. Although these problems may be few and far between, it may be important to include error testing in the future to guard against similar SDF atom block ambiguities during the molecule and topology loading process.
## Testing Steps

The following tests were run to compare two parallel lists of `openff.toolkit.molecule.Molecule` objects.

1. comparing the output of `Molecule.to_smiles` 
    1. return 1 if smiles are identical, 0 if smiles are similar, -1 else
2. comparing the output of `Molecule.to_inchi`
    1. return 3 if inchi are identical, 2, 1, 0, or -1 else
3. comparing the results of `Molecule.are_isomorphic` with 3 different bond matching functions:
    1. match by aromaticity only `Molecule.are_isomorphic(mol1, mol2, bond_order_matching=False)`
    2. match by bond order only `Molecule.are_isomorphic(mol1, mol2, aromatic_matching=False)`
    3. match by both `Molecule.are_isomorphic(mol1, mol2)`
4. comparing the explicit hydrogen and heavy atoms as a double check on the previous tests. 

<img src="3d_compare_flow_chart_new.png" alt="Drawing" style="width: 900px;"/>

## Results

Errors can be placed into the following categories:

1. Different kekule structures around aromatic rings (Low Priority)
2. OpenEye including one or more extra protons on ambiguous molecules (High Priority)
3. Different stereochemistry in the sulfur of solfoxides (High Priority)
4. Difference in stereochemistry around pyramidal nitrogens (Low Priority - already accounted for) 
5. RDKit refusing to load in nitrogens with 4 bonds and ambiguous formal charge (Medium Priority)


# Loading all Required Libraries

In [1]:
import warnings
from copy import deepcopy
import os
from pathlib import Path

import networkx as nx
import numpy as np
from networkx.algorithms.isomorphism import GraphMatcher
from simtk import unit
from simtk.openmm.app import Element, element

import openff.toolkit
from openff.toolkit.utils import (
    MessageException,
    check_units_are_compatible,
    deserialize_numpy,
    quantity_to_string,
    serialize_numpy,
    string_to_quantity,
)
from openff.toolkit.utils.serialization import Serializable
from openff.toolkit.utils.toolkits import (
    DEFAULT_AROMATICITY_MODEL,
    GLOBAL_TOOLKIT_REGISTRY,
    InvalidToolkitRegistryError,
    OpenEyeToolkitWrapper,
    RDKitToolkitWrapper,
    ToolkitRegistry,
    ToolkitWrapper,
    UndefinedStereochemistryError,
)

from openff.toolkit.utils.toolkits import RDKitToolkitWrapper, OpenEyeToolkitWrapper, UndefinedStereochemistryError
from openff.toolkit.utils import get_data_file_path
from openff.toolkit.topology.molecule import Molecule, FrozenMolecule
import pandas as pd
import difflib
import matplotlib.pyplot as plt
import matplotlib as mpl

# following is needed so that the child classes bellow run with all the libraries found in toolkits.py
import logging 
logger = logging.getLogger(__name__)

# Define smiles and inchi Comparison Functions
`are_isomorphic()` is already included in the toolkit Molecule. These two functions follow the same form as are_isomorphic: `comparison_function(off_molecule_1, off_molecule_2, toolkit_wrapper)`. The `toolkit_wrapper` can be used to specify which toolkit is used when converting the molecule to a smiles or inchi string. Two helper functions were also used to help catch and identify any differences. 

In [2]:
# 2) Function 1 - OFF to SMILES string then string comparison
def get_str_symbols(string):
    """
    returns a dictionary with the symbols as keys and the number of occurences of each symbol as the values from str. 
    This function does not work well for formulas with Ni, Ts, Nb, and some other elements that will not ever be used
    """
    letters = dict()
    ignore = "[]()123456789-+=\?#$:.Hh@"
    for i in range(len(string)):
        f = string[i:i+1]
        n = string[i+1:i+2]
        if f in ignore:
            continue
        if f.isupper():
            if n in 'ltre':
                i_str = f + n
            else:
                i_str = f
        elif f in 'bcnops':
            i_str = f
        else:
            continue

        if i_str in letters.keys():
            letters[i_str] += 1
        else:
            letters[i_str] = 1

    return letters

def smiles_str_compare(mol1, mol2, toolkit_wrapper, isom=True, explicit_h=True):
    """
    converts the molecules to a SMILES string and then does a string comparison 
    mol1: an OFF Molecule class 
    mol2: an OFF Molecule class
    toolkit_wrapper: either RDKitToolkitWrapper or OpenEyeToolkitWrapper (hopefully testing will show that this distinction does not matter)
    returns: 1 if SMILES are identical, 0 if SMILES have the same number of heavy atoms (everything except H), -1 else 
    """
    try:
        mol1_str = mol1.to_smiles(toolkit_registry=toolkit_wrapper, isomeric=isom, explicit_hydrogens=explicit_h)
        mol2_str = mol2.to_smiles(toolkit_registry=toolkit_wrapper, isomeric=isom, explicit_hydrogens=explicit_h)

        if mol1_str == mol2_str:
            return 1
        # now we start the comparison of all the heavy atoms. Hopefully this will tell us if explicit H's are the cause of the string inequality
        mol1_symbols = get_str_symbols(mol1_str)
        mol2_symbols = get_str_symbols(mol2_str)
        if mol1_symbols == mol2_symbols:
            return 0

        return -1
    except Exception:
        return -999

# 3) Function 2 - OFF to InChI string then string comparison
def parse_std_InChI(string):
    """
    Does a basic parsing of a standard InChI string using what rules I currently know.
    Returns [atom_block, connectivity_block, explicity_hydrogen_block]
    """

    layers = string.split('/')
    atoms = None
    connect = None
    hydro = None
    for layer in layers:
        if 'InChI=' in layer:
            continue    
        if layer[0:1].isupper():
            atoms = layer
        if layer[0:1] == 'c':
            connect = layer
        if layer[0:1] == 'h':
            hydro = layer
    return [atoms, connect, hydro]
    
def inchi_str_compare(mol1, mol2, toolkit_wrapper, fixed_h = False):
    """
        converts the molecules to an InChI string and then does a string comparison 
        mol1: an OFF Molecule class 
        mol2: an OFF Molecule class
        toolkit_wrapper: either RDKitToolkitWrapper or OpenEyeToolkitWrapper (hopefully testing will show that this distinction does not matter)
        returns: 3 if InChI are identical, 
                 2 if InChI have the same atoms and connectivity and explicit hydrogen block, 
                 1 if InChI have the same atoms and connectivity
                 0 if InChI have the same atoms
                 -1 else
    """
    try:
        mol1_str = mol1.to_inchi(fixed_hydrogens=fixed_h, toolkit_registry=toolkit_wrapper)
        mol2_str = mol2.to_inchi(fixed_hydrogens=fixed_h, toolkit_registry=toolkit_wrapper)

        if mol1_str == mol2_str:
            return 3
        # now we start the comparison of all the heavy atoms. Hopefully this will tell us if explicit H's are the cause of the string inequality
        mol1_blocks = parse_std_InChI(mol1_str)
        mol2_blocks = parse_std_InChI(mol2_str)
        if mol1_blocks[0] == mol2_blocks[0]:  #compares the atoms block
            if mol1_blocks[1] == mol2_blocks[1]:  #compares the connectivity block
                if mol1_blocks[2] == mol2_blocks[2]:  #compares the explicity hydrgen block
                    return 2
                return 1
            return 0
        else:
            return -1
    except Exception:
        return -999

# Getting Ready to Load in Molecules
The following class definitions inherit directly from `Molecule`. This is only for testing purposes. The basic functionality of the class is unchanged, but I have added some try-except clauses to catch any errors (since the goal of this entire notebook is to break existing functionality anyway). This also provides a good framework in general for making any future testing changes to the toolkit wrappers. 


In [3]:
class TestRDKitToolkitWrapper(RDKitToolkitWrapper):
    # An exact copy from original except for some minor changes:
    # When a file is not read correctly, instead of continuing to the next molecule, append a "None" type 
    # to the mols list. This ensures that both Toolkits return exactly 371 things, even if some are "None" 
    def from_file(
        self, file_path, file_format, allow_undefined_stereo=False, _cls=None
    ):
        from rdkit import Chem

        file_format = file_format.upper()

        mols = list()
        if (file_format == "MOL") or (file_format == "SDF"):
            for rdmol in Chem.SupplierFromFilename(
                file_path, removeHs=False, sanitize=False, strictParsing=True
            ):
                if rdmol is None:
                    mols.append(np.NaN)     # --------------------------------------------> added from original
                    continue

                # Sanitize the molecules (fails on nitro groups)
                try:
                    Chem.SanitizeMol(
                        rdmol,
                        Chem.SANITIZE_ALL
                        ^ Chem.SANITIZE_SETAROMATICITY
                        ^ Chem.SANITIZE_ADJUSTHS,
                    )
                    Chem.AssignStereochemistryFrom3D(rdmol)
                except ValueError as e:
                    logger.warning(rdmol.GetProp("_Name") + " " + str(e))
                    mols.append(np.NaN)   #-------------------------------------------------> added from original
                    continue
                Chem.SetAromaticity(rdmol, Chem.AromaticityModel.AROMATICITY_MDL)
                mol = self.from_rdkit(
                    rdmol, allow_undefined_stereo=allow_undefined_stereo, _cls=_cls
                )
                mols.append(mol)

        elif file_format == "SMI":
            # TODO: We have to do some special stuff when we import SMILES (currently
            # just adding H's, but could get fancier in the future). It might be
            # worthwhile to parse the SMILES file ourselves and pass each SMILES
            # through the from_smiles function instead
            for rdmol in Chem.SmilesMolSupplier(file_path, titleLine=False):
                rdmol = Chem.AddHs(rdmol)
                mol = self.from_rdkit(
                    rdmol, allow_undefined_stereo=allow_undefined_stereo, _cls=_cls
                )
                mols.append(mol)

        elif file_format == "PDB":
            raise Exception(
                "RDKit can not safely read PDBs on their own. Information about bond order and aromaticity "
                "is likely to be lost. To read a PDB using RDKit use Molecule.from_pdb_and_smiles()"
            )

        return mols
class TestOpenEyeToolkitWrapper(OpenEyeToolkitWrapper):
    # unchanged 
    def from_file(
        self, file_path, file_format, allow_undefined_stereo=False, _cls=None
    ):
        from openeye import oechem

        ifs = oechem.oemolistream(file_path)
        return self._read_oemolistream_molecules(
            ifs, allow_undefined_stereo, file_path=file_path, _cls=_cls
        )
class TestMolecule(Molecule):
    # this is the exact same function but wrapped in a try statement to help debug 
    @staticmethod
    def are_isomorphic(
        mol1,
        mol2,
        return_atom_map=False,
        aromatic_matching=True,
        formal_charge_matching=True,
        bond_order_matching=True,
        atom_stereochemistry_matching=True,
        bond_stereochemistry_matching=True,
        strip_pyrimidal_n_atom_stereo=True,
        toolkit_registry=GLOBAL_TOOLKIT_REGISTRY,
    ):

        try:
            # Do a quick hill formula check first
            if Molecule.to_hill_formula(mol1) != Molecule.to_hill_formula(mol2):
                return False, None

            # Build the user defined matching functions
            def node_match_func(x, y):
                # always match by atleast atomic number
                is_equal = x["atomic_number"] == y["atomic_number"]
                if aromatic_matching:
                    is_equal &= x["is_aromatic"] == y["is_aromatic"]
                if formal_charge_matching:
                    is_equal &= x["formal_charge"] == y["formal_charge"]
                if atom_stereochemistry_matching:
                    is_equal &= x["stereochemistry"] == y["stereochemistry"]
                return is_equal

            # check if we want to do any bond matching if not the function is None
            if aromatic_matching or bond_order_matching or bond_stereochemistry_matching:

                def edge_match_func(x, y):
                    # We don't need to check the exact bond order (which is 1 or 2)
                    # if the bond is aromatic. This way we avoid missing a match only
                    # if the alternate bond orders 1 and 2 are assigned differently.
                    if aromatic_matching and bond_order_matching:
                        is_equal = (x["is_aromatic"] == y["is_aromatic"]) or (
                            x["bond_order"] == y["bond_order"]
                        )
                    elif aromatic_matching:
                        is_equal = x["is_aromatic"] == y["is_aromatic"]
                    elif bond_order_matching:
                        is_equal = x["bond_order"] == y["bond_order"]
                    else:
                        is_equal = None
                    if bond_stereochemistry_matching:
                        if is_equal is None:
                            is_equal = x["stereochemistry"] == y["stereochemistry"]
                        else:
                            is_equal &= x["stereochemistry"] == y["stereochemistry"]

                    return is_equal

            else:
                edge_match_func = None

            # Here we should work out what data type we have, also deal with lists?
            def to_networkx(data):
                """For the given data type, return the networkx graph"""
                from openff.toolkit.topology import TopologyMolecule

                if strip_pyrimidal_n_atom_stereo:
                    SMARTS = "[N+0X3:1](-[*])(-[*])(-[*])"

                if isinstance(data, FrozenMolecule):
                    # Molecule class instance
                    if strip_pyrimidal_n_atom_stereo:
                        # Make a copy of the molecule so we don't modify the original
                        data = deepcopy(data)
                        data.strip_atom_stereochemistry(
                            SMARTS, toolkit_registry=toolkit_registry
                        )
                    return data.to_networkx()
                elif isinstance(data, TopologyMolecule):
                    # TopologyMolecule class instance
                    if strip_pyrimidal_n_atom_stereo:
                        # Make a copy of the molecule so we don't modify the original
                        ref_mol = deepcopy(data.reference_molecule)
                        ref_mol.strip_atom_stereochemistry(
                            SMARTS, toolkit_registry=toolkit_registry
                        )
                    return ref_mol.to_networkx()
                elif isinstance(data, nx.Graph):
                    return data

                else:
                    raise NotImplementedError(
                        f"The input type {type(data)} is not supported,"
                        f"please supply an openff.toolkit.topology.molecule.Molecule,"
                        f"openff.toolkit.topology.topology.TopologyMolecule or networkx "
                        f"representation of the molecule."
                    )

            mol1_netx = to_networkx(mol1)
            mol2_netx = to_networkx(mol2)
            from networkx.algorithms.isomorphism import GraphMatcher

            GM = GraphMatcher(
                mol1_netx, mol2_netx, node_match=node_match_func, edge_match=edge_match_func
            )
            isomorphic = GM.is_isomorphic()

            if isomorphic and return_atom_map:
                topology_atom_map = GM.mapping

                # reorder the mapping by keys
                sorted_mapping = {}
                for key in sorted(topology_atom_map.keys()):
                    sorted_mapping[key] = topology_atom_map[key]

                return isomorphic, sorted_mapping

            else:
                return isomorphic, None
        except Exception:
            return -99

# Loading in the Molecules with Errors included

In [4]:

drug_bank_file = get_data_file_path('molecules/MiniDrugBank.sdf')
rdkit_toolkit_wrapper = TestRDKitToolkitWrapper()
open_eye_toolkit_wrapper = TestOpenEyeToolkitWrapper()

OFF_using_rd = Molecule.from_file(drug_bank_file, 
                                toolkit_registry=rdkit_toolkit_wrapper, 
                                file_format='sdf', 
                                allow_undefined_stereo=True)
OFF_using_oe = Molecule.from_file(drug_bank_file, 
                                toolkit_registry=open_eye_toolkit_wrapper, 
                                file_format='sdf', 
                                allow_undefined_stereo=True)
assert(len(OFF_using_rd) == len(OFF_using_oe))

mols = pd.DataFrame(data={"OFF_from_OE": OFF_using_oe, "OFF_from_RD": OFF_using_rd})
# perform a basic check to make sure the dataframe lines up. This is just for my sanity

all_are_the_same = 1

[(all_are_the_same * int((str(mol1.name)==str(mol2.name)))) for mol1, mol2 in zip(mols['OFF_from_OE'], mols['OFF_from_RD']) if not isinstance(mol2,float)]
# if there are any differences (besides RDKit molecules being np.NaN due to incorrect loading that I don't know how to fix right now), 
# all_are_the_same will return as 0. 
assert(all_are_the_same == 1)

RDKit ERROR: [11:54:55] Explicit valence for atom # 21 N, 4, is greater than permitted
DrugBank_2799 Explicit valence for atom # 21 N, 4, is greater than permitted
RDKit ERROR: [11:54:55] Explicit valence for atom # 12 N, 4, is greater than permitted
DrugBank_5415 Explicit valence for atom # 12 N, 4, is greater than permitted
RDKit ERROR: [11:54:55] Explicit valence for atom # 3 N, 4, is greater than permitted
DrugBank_3046 Explicit valence for atom # 3 N, 4, is greater than permitted
RDKit ERROR: [11:54:55] Explicit valence for atom # 16 N, 4, is greater than permitted
DrugBank_472 Explicit valence for atom # 16 N, 4, is greater than permitted
Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. RDMol name: DrugBank_5847Undefined chiral centers are:
 - Atom C (index 2)

RDKit ERROR: [11:54:55] Explicit valence for atom # 10 N, 4, is greater than permitted
DrugBank_794 Explicit valence for atom # 10 N, 4, is g

# Running the comparisons

In [5]:
def unwrapper_func(are_isomorphic_output):
    """
    This is a helper function that helps unwrap the output from the are_isomorphic function in the TestMolecule class
    """
    if are_isomorphic_output == -99:
        return -99
    else:
        f, l = are_isomorphic_output
        return f
def compare_func(mol1, mol2):
    """
    another helper function to assist in the comparisons. In future, a regular for loop would be much more readable
    """
    if isinstance(mol1, Molecule) and isinstance(mol2, Molecule):
        return True
    else:
        if isinstance(mol2, float):
            if mol2 != mol2:
                return False
            elif mol2 < -10:
                return True
            else:
                print("unexplected case in compare_func")
                return False
        elif isinstance(mol1, float):
            if mol1 != mol1:
                return False
            elif mol1 < -10:
                return True
            else:
                print("unexplected case in compare_func")
                return False

    

comparison_results = pd.DataFrame(columns=['name', 
                                           'smi_rd', 
                                           'smi_oe', 
                                           'inch_rd', 
                                           'inch_oe', 
                                           'iso_arom', 
                                           'iso_bond_order', 
                                           'iso_arom_bond_order'])
comparison_results['name'] = [mol.name for mol in mols['OFF_from_OE']]

comparison_results['smi_rd'] = [smiles_str_compare(mol_oe, mol_rd, toolkit_wrapper=rdkit_toolkit_wrapper) \
    if (compare_func(mol_oe, mol_rd)) else "nan" \
        for mol_oe, mol_rd in zip(mols['OFF_from_OE'], mols['OFF_from_RD'])]
print("1")

comparison_results['smi_oe'] = [smiles_str_compare(mol_oe, mol_rd, toolkit_wrapper=open_eye_toolkit_wrapper) \
    if (compare_func(mol_oe, mol_rd)) else "nan" \
        for mol_oe, mol_rd in zip(mols['OFF_from_OE'], mols['OFF_from_RD'])]
print("2")
comparison_results['inch_rd'] = [inchi_str_compare(mol_oe, mol_rd, toolkit_wrapper=rdkit_toolkit_wrapper) \
    if (compare_func(mol_oe, mol_rd)) else "nan" \
        for mol_oe, mol_rd in zip(mols['OFF_from_OE'], mols['OFF_from_RD'])]
print("3")
comparison_results['inch_oe'] = [inchi_str_compare(mol_oe, mol_rd, toolkit_wrapper=open_eye_toolkit_wrapper) \
    if (compare_func(mol_oe, mol_rd)) else "nan" \
        for mol_oe, mol_rd in zip(mols['OFF_from_OE'], mols['OFF_from_RD'])]
print("4")

comparison_results['iso_arom'] = [unwrapper_func(TestMolecule.are_isomorphic(mol_oe, mol_rd, bond_order_matching=False)) \
    if (compare_func(mol_oe,mol_rd)) else "nan" \
        for mol_oe, mol_rd in zip(mols['OFF_from_OE'], mols['OFF_from_RD'])]
print("5")
comparison_results['iso_bond_order'] = [unwrapper_func(TestMolecule.are_isomorphic(mol_oe, mol_rd, aromatic_matching=False)) \
    if (compare_func(mol_oe, mol_rd)) else "nan" \
        for mol_oe, mol_rd in zip(mols['OFF_from_OE'], mols['OFF_from_RD'])]
print("6")
comparison_results['iso_arom_bond_order'] = [unwrapper_func(TestMolecule.are_isomorphic(mol_oe, mol_rd)) \
    if (compare_func(mol_oe, mol_rd)) else "nan" \
        for mol_oe, mol_rd in zip(mols['OFF_from_OE'], mols['OFF_from_RD'])]
print("7")

1
2
3
4
5
6
7


In [6]:
print(comparison_results)
comparison_results.to_csv("comparison_results.csv") # for viewing with excel, table, etc. 

              name smi_rd smi_oe inch_rd inch_oe iso_arom iso_bond_order  \
0    DrugBank_5354      1      1       3       3     True           True   
1    DrugBank_2791      1      1       3       3     True           True   
2    DrugBank_5373      1      1       3       3     True           True   
3    DrugBank_2799    nan    nan     nan     nan      nan            nan   
4    DrugBank_2800      1      1       3       3     True           True   
..             ...    ...    ...     ...     ...      ...            ...   
366  DrugBank_2728      1      1       3       3     True           True   
367  DrugBank_5319      1      1       3       3     True           True   
368  DrugBank_5323      1      1       3       3     True           True   
369  DrugBank_5326      1      1       3       3     True           True   
370  DrugBank_5329      1      1       3       3     True           True   

    iso_arom_bond_order  
0                  True  
1                  True  
2        

# Manual Atom Comparisons
The following comparisons are not strictly necessary but still provide usefull info on atom counts. The general idea is to manually count all explicit hydrogen and heavy atoms, store this information in a dictionary, and compare the results. This is a welcome alternative to loading in the original sdf file and manually counting the number of protons in all failure molecules to test for protonization. Below, 

In [7]:
class CustomSDFMolSupplier:
    """
    returns each individual SDF block from an SDF file. So far, only tested and working on
    MiniDrugBank.sdf. With the nature of the file parsing, errors are extremely likely. It would
    be nice if I could get ahold of some "official" code that does this.
    """
    def __init__(self, file):
        
        self.file = self.check_file(file)
        self.at_end_of_file = False
        self.file_length = sum(1 for line in open(file, "r"))
        self.is_sdf()  # checks for file identity
        self.current_mol = 0

    def check_file(self, file):
        try:
            from pathlib import Path
        except Exception as e:
            print(f"Exception during import in CustomSDFMolSupplier: \n\n {e} \n")
        if isinstance(file, str):
            f = Path(file)
        elif isinstance(file, Path):
            f = file
        else:
            print(f"error in CustomSDFMolSupplier:\n\t type {type(file)} given, which is not accepted. Use a string or Path variable")
        return f    
    
    def is_sdf(self):
        if self.file.suffix != '.sdf':
            raise NameError("sdf file required for class \"CustomSDFMolSupplier\"")

    def get_next_block(self):
        """
        read from the current file index "curr" to the start of the next molecule
        """
        f = open(self.file, "r")

        sdfstring = ""     # this is only defined here to that the top-most molecule does not break
        for i, line in enumerate(f):
            # parse through until the correct line has been reached
            if i < self.current_mol:
                continue
            # if we have reached the end of the current molecule, break
            if line == "$$$$\n":
                if (i + 1 == self.file_length): # if line is the last line of the file
                    self.at_end_of_file = True
                    self.current_mol = i + 1
                    break
                elif i == self.current_mol:   #if at start of the molecule
                    sdfstring = ""
                    continue
                else:                       #if at the end
                    self.current_mol = i
                    break
            if i == self.current_mol + 1 and i != 1:   #get the identifier of the molecule
                name = line.split("\n")[0]  #remove newline 
            elif i == 0:         # special case when reading the first block of the file
                name = line.split("\n")[0]
            sdfstring = "".join((sdfstring, line))
        return name, sdfstring

    def __iter__(self):
        self.current_mol = 0
        self.at_end_of_file = False
        return self
    def __next__(self):
        if self.at_end_of_file:
            raise StopIteration
        mol_name, mol2_block = self.get_next_block()
        return mol_name, mol2_block

class BlockToAtomSupplier(CustomSDFMolSupplier):
    """
    Does the same thing as CustomSDFMolSupplier but returns atom info instead of the raw SDF block
    Same applies here - lots of testing needs to be done, but these classes work for now.
    """
    def get_str_symbols(self, string):
        """
        returns a dictionary with the symbols as keys and the number of occurences of each symbol as the values from str. 
        This function does not work well for formulas with Ni, Ts, Nb, and some other elements that will not ever be used
        """
        letters = dict()
        ignore = "[]()123456789-+=\?#$:.@"
        for i in range(len(string)):
            f = string[i:i+1]
            n = string[i+1:i+2]
            if f in ignore:
                continue
            if f.isupper():
                if n in 'ltre':
                    i_str = f + n
                else:
                    i_str = f
            elif f in 'bcnops':
                i_str = f
            else:
                continue

            if i_str in letters.keys():
                letters[i_str] += 1
            else:
                letters[i_str] = 1

        return letters

    def parse_atoms(self, b):
        """
        returns a dictionary of atom names (key) and the corresponding number of atoms of that type.
        If you run into issues with this function, it probably means that the sdf block or file you are reading
        this from differs from MiniDrugBank.sdf by a single space or indetation difference. 
        """
        string = ""
        split_array = b.split("\n")
        found = False
        for line in split_array: # only want to get the "big block" of info
            if len(line) > 60:
                found = True
                string += line
            if len(line) < 40 and found:  # stop reading the rest when done 
                break
        return self.get_str_symbols(string)

    def __next__(self):
        if self.at_end_of_file:
            raise StopIteration
        mol_name, mol2_block = self.get_next_block()
        return mol_name, self.parse_atoms(mol2_block)

def get_atom_nums(mols):
    """
    This func takes in mols and returns a generator object that supplies dictionaries with element symbols as keys and number of atoms for each element as the values.
    Mols is a list of OFF Molecules. Can also accept a pandas series. stops iteration and prints error if error occurs 
    """
    # begin with list and Series handling
    try:
        if isinstance(mols, list):
            pass
        elif isinstance(mols, pd.Series):
            mols = mols.to_list()
        else:
            print(f"improper Dtype <{type(mols)}> passed to get_atom_nums(mols)")
            raise StopIteration
    except Exception:
        print(f"improper Dtype <{type(mols)}> passed to get_atom_nums(mols)")
        raise StopIteration
    # define the function that operates on an individual OFFMol and returns a dictionary

    def OFF_dict(mol):
        atom_syms = [atom.element.symbol for atom in mol.atoms]
        atom_dict = dict()
        for a in atom_syms:
            if a in atom_dict.keys():
                atom_dict[a] += 1
            else:
                atom_dict[a] = 1
        return atom_dict

    for mol in mols:
        try:
            yield mol.name, OFF_dict(mol)
        except Exception:
            yield -999, "tears"

# Running the Manual Comparisons

In [8]:
manual_comparison_results = pd.DataFrame(columns=['name2', 'manual_reading', 'open_eye', 'rdkit', 'proton_num(man vs oe vs rd)','comparison'])
for file_results, off_oe, off_rd in zip(BlockToAtomSupplier(drug_bank_file), get_atom_nums(mols['OFF_from_OE']), get_atom_nums(mols['OFF_from_RD'])):
    manual_name, manual_dict = file_results
    oe_name, oe_results = off_oe
    rd_name, rd_results = off_rd
    if manual_name == oe_name == rd_name:
        are_same = (manual_dict == oe_results == rd_results)
        row = pd.Series({'name2': manual_name, 
                'manual_reading': manual_dict, 
                'open_eye': oe_results, 
                'rdkit': rd_results, 
                'proton_num(man vs oe vs rd)': f"{manual_dict.get('H')} vs {oe_results.get('H')} vs {rd_results.get('H')}",
                'comparison': are_same})
        manual_comparison_results = manual_comparison_results.append(row, ignore_index=True)
    else:
        row = pd.Series({'name2': manual_name, 
                'manual_reading': "--", 
                'open_eye': "--", 
                'rdkit': "--", 
                'proton_num(man vs oe vs rd)': "--",
                'comparison': "--"})
        manual_comparison_results = manual_comparison_results.append(row, ignore_index=True)
        
manual_comparison_results.to_csv("manual_comparison_results.csv") # for excel or table reading
print('all manual comparison differences:')
print(manual_comparison_results[['name2', 'proton_num(man vs oe vs rd)', 'comparison']].loc[manual_comparison_results['comparison'] == False])

all manual comparison differences:
             name2 proton_num(man vs oe vs rd) comparison
11   DrugBank_5418              14 vs 15 vs 14      False
18    DrugBank_178              21 vs 22 vs 21      False
26    DrugBank_246              17 vs 18 vs 17      False
71   DrugBank_5847              36 vs 37 vs 36      False
77    DrugBank_700              25 vs 26 vs 25      False
143  DrugBank_3930                 5 vs 7 vs 5      False
184  DrugBank_1564              10 vs 14 vs 10      False
200  DrugBank_1634              21 vs 23 vs 21      False
211  DrugBank_1700              14 vs 17 vs 14      False
261  DrugBank_1962              36 vs 37 vs 36      False
272  DrugBank_4662              21 vs 23 vs 21      False
282  DrugBank_2052              15 vs 19 vs 15      False
287  DrugBank_2077           None vs 2 vs None      False
289  DrugBank_2082              16 vs 19 vs 16      False
297  DrugBank_2148                 3 vs 4 vs 3      False
305  DrugBank_2210              18 vs

# Organizing the Results
So far, I have 2 dataframes (comparison_results and manual_comparison_results). The following block organizes the data and categorizes the data according to a number of observations I have gathered while looking at the data. The general categories of comparison errors are:

1. arom_kekule_diff: difference in the kekule structures around aromatic rings. These are caught by the are_isomorphic function
2. stereo_diff: difference in stereochemistry. These are caught when molecules have the same number of atoms/protons but return False from are_isomorphic
3. proton_diff: difference in proton counts. These are caught when the manual comparison returns False and sometimes when the InChI comparison fails
4. failed_to_load: These include molecules that break RDKit. OpenEye is still able to load in molecules of this type.

In [9]:
df1 = comparison_results
df2 = manual_comparison_results
combined = pd.concat([mols, df1, df2], axis=1)
# The following logic may seem arbitrary but makes more sense upon closer inspection of the data from comparison_results.csv and manual_comparison_results.csv
condition = {'arom_kekule_diff': (combined['comparison'] == True) & (combined['iso_bond_order'] == False) & (combined['iso_arom'] == True),
            'stereo_diff': (combined['comparison'] == True) & (combined['iso_bond_order'] == False) & (combined['iso_arom'] == False),
            'proton_diff': (combined['comparison'] == False), 
            'failed_to_load': (combined['smi_rd'] == 'nan'),
            'smiles_diff': ((combined['smi_rd'] != 1) | (combined['smi_oe'] != 1)) & (combined['comparison'] == True),  # ---> only want smi/inch differences that haven't
            'inchi_diff': ((combined['inch_rd'] != 3) | (combined['inch_oe'] != 3)) & (combined['comparison'] == True)}   # ---> already been caught by manual comparison
for error_type in condition.keys():
    combined[error_type] = condition.get(error_type)
# check one last time to make sure all names match. Not really necessary but oh well... 
for i, row in combined.iterrows():
    if (not isinstance(row['OFF_from_RD'], float)) and (row['name2'] == row['name'] == row['OFF_from_OE'].name == row['OFF_from_RD'].name):
        continue
    elif (isinstance(row['OFF_from_RD'], float)) and (row['name2'] == row['name']):   # as a reminder, these cases indicate that RDKit failed to load in the molecule 
        continue 
    else:
        print('critical error in organize_errors: names do not match')
        raise Exception

# The only data of interest are the molecules that cause errors:
combined = combined[combined['arom_kekule_diff'] | combined['stereo_diff'] | combined['proton_diff'] | combined['failed_to_load'] | combined['smiles_diff'] | combined['inchi_diff']]


# Vizuallizing the errors
PyMol and NetworkX (combined with matplotlib) were used to vizuallize the Molecules and spot any sources of possible confusion. I chose to use the networkx graphing method over using the existing openeye and rdkit plotting functionality to avoid any more convertion between openeye and rdkit. 
## Creating Networkx Plots

In [10]:
def graph_networkx(mol1, folder, toolkit_wrapper, strip_pyrimidal_n_atom_stereo=False, file_id=""):
    # the following function comes straight from the are_isomorphic function from the FrozenMolecule class (see line 2595 molecule.py)
    def to_networkx(data):
            """For the given data type, return the networkx graph"""

            if strip_pyrimidal_n_atom_stereo:
                SMARTS = "[N+0X3:1](-[*])(-[*])(-[*])"

            if isinstance(data, FrozenMolecule):
                # Molecule class instance
                if strip_pyrimidal_n_atom_stereo:
                    # Make a copy of the molecule so we don't modify the original
                    data = deepcopy(data)
                    data.strip_atom_stereochemistry(
                        SMARTS, toolkit_registry=toolkit_wrapper
                    )
                return data.to_networkx()

            else:
                raise NotImplementedError(
                    f"The input type {type(data)} is not supported,"
                    f"please supply an openff.toolkit.topology.molecule.Molecule,"
                )

    mol1_netx = to_networkx(mol1)

    color_map = {1: 'y',
                6: 'k',
                7: 'b',
                8: 'r',
                15: 'm',
                16: 'g'
                }
    size_map = {1: 20,
                6: 75,
                7: 150,
                8: 150,
                15: 200,
                16: 200
                }
    label_map = {1: "",
                6: "",
                7: "N",
                8: "O",
                15: "P",
                16: "S"
                }
    labeldict = dict()
    for i, val in mol1_netx.nodes(data=True):
        labeldict[i] = label_map.get(val.get('atomic_number'), f"{val.get('atomic_number')}")

    color_values = [color_map.get(val.get('atomic_number'), 'c') for i, val in mol1_netx.nodes(data=True)]
    size_values = [size_map.get(val.get('atomic_number'), 200) for i, val in mol1_netx.nodes(data=True)]
    single_bonds = [edge for edge in mol1_netx.edges(data=True) if (edge[2].get('bond_order') == 1)]
    double_bonds = [edge for edge in mol1_netx.edges(data=True) if (edge[2].get('bond_order') == 2)]
    triple_bonds = [edge for edge in mol1_netx.edges(data=True) if (edge[2].get('bond_order') == 3)]

    plt.clf()
    fig, ax = plt.subplots(1, 1, num=1)

    pos = nx.kamada_kawai_layout(mol1_netx)
    nx.draw_networkx_nodes(mol1_netx, pos, node_color = color_values, node_size = size_values, ax=ax)
    nx.draw_networkx_labels(mol1_netx, pos, font_size = 7, alpha = 0.75, ax=ax, labels=labeldict)
    nx.draw_networkx_edges(mol1_netx, pos, edgelist=single_bonds, edge_color='k', ax=ax)
    nx.draw_networkx_edges(mol1_netx, pos, edgelist=double_bonds, edge_color='r', ax=ax)
    nx.draw_networkx_edges(mol1_netx, pos, edgelist=triple_bonds, edge_color='c', ax=ax)
    plt.savefig(f"netx_figs/{folder}/{mol1.name}({file_id}).png", dpi=600)
    plt.clf()


def graph_mult_networkx(mol_list, name_list, toolkit_wrapper, folder, strip_pyrimidal_n_atom_stereo=False, file_id=""):
    # the following function comes straight from the are_isomorphic function from the FrozenMolecule class (see line 2595 molecule.py)
    def to_networkx(data):
            """For the given data type, return the networkx graph"""

            if strip_pyrimidal_n_atom_stereo:
                SMARTS = "[N+0X3:1](-[*])(-[*])(-[*])"

            if isinstance(data, FrozenMolecule):
                # Molecule class instance
                if strip_pyrimidal_n_atom_stereo:
                    # Make a copy of the molecule so we don't modify the original
                    data = deepcopy(data)
                    data.strip_atom_stereochemistry(
                        SMARTS, toolkit_registry=toolkit_wrapper
                    )
                return data.to_networkx()

            else:
                raise NotImplementedError(
                    f"The input type {type(data)} is not supported,"
                    f"please supply an openff.toolkit.topology.molecule.Molecule,"
                )

    filename = ""
    for mol in mol_list:
        filename = filename + mol.name + "_"

    color_map = {1: 'y',
                6: 'k',
                7: 'b',
                8: 'r',
                15: 'm',
                16: 'g'
                }
    size_map = {1: 20,
                6: 75,
                7: 150,
                8: 150,
                15: 200,
                16: 200
                }
    label_map = {1: "",
                6: "",
                7: "N",
                8: "O",
                15: "P",
                16: "S"
                }
    plt.clf()
    fig, ax = plt.subplots(len(mol_list), 1, num=1)
    plot = 0
    for mol in mol_list:
        mol_netx = to_networkx(mol)

        labeldict = dict()
        for i, val in mol_netx.nodes(data=True):
            labeldict[i] = label_map.get(val.get('atomic_number'), f"{val.get('atomic_number')}")

        color_values = [color_map.get(val.get('atomic_number'), 'c') for i, val in mol_netx.nodes(data=True)]
        size_values = [size_map.get(val.get('atomic_number'), 200) for i, val in mol_netx.nodes(data=True)]
        single_bonds = [edge for edge in mol_netx.edges(data=True) if (edge[2].get('bond_order') == 1)]
        double_bonds = [edge for edge in mol_netx.edges(data=True) if (edge[2].get('bond_order') == 2)]
        triple_bonds = [edge for edge in mol_netx.edges(data=True) if (edge[2].get('bond_order') == 3)]
        
        pos = nx.kamada_kawai_layout(mol_netx)

        nx.draw_networkx_nodes(mol_netx, pos, node_color = color_values, node_size = size_values, ax=ax[plot])
        nx.draw_networkx_labels(mol_netx, pos, font_size = 7, alpha = 0.75, ax=ax[plot], labels=labeldict)
        nx.draw_networkx_edges(mol_netx, pos, edgelist=single_bonds, edge_color='k', ax=ax[plot])
        nx.draw_networkx_edges(mol_netx, pos, edgelist=double_bonds, edge_color='r', ax=ax[plot])
        nx.draw_networkx_edges(mol_netx, pos, edgelist=triple_bonds, edge_color='c', ax=ax[plot])
        mpl.rcParams['text.color'] = 'c'
        ax[plot].title.set_text(name_list[plot])

        plot += 1
    plt.savefig(f"netx_figs/{folder}/{filename}({file_id}).png", dpi=600)
    plt.clf()

# create a folder for the figs 
curr = Path.cwd()
if curr.name != '3d_equivalence_testing_final':
    raise Exception
# the following directories will show up in a subfolder under /netx_figs
error_dirs = ['arom_kekule_diff',
            'stereo_diff',
            'proton_diff',
            'failed_to_load',
            'smiles_diff',
            'inchi_diff']
for dir in error_dirs:
    try:
        (curr / "netx_figs" / dir).mkdir(parents=True, exist_ok=False)
    except FileExistsError:
        print("Folder is already there")
    else:
        print("Folder was created")

for folder in error_dirs:
    error_mols = combined.loc[combined[folder], ['OFF_from_OE', 'OFF_from_RD']]
    for i, mol1, mol2 in zip(error_mols.index, error_mols['OFF_from_OE'], error_mols['OFF_from_RD']):
        if isinstance(mol1, Molecule) and isinstance(mol2, Molecule):
            if mol1.name != mol2.name:
                print("\n\n\n\nCritical Error in graph_deviants")
                raise Exception
            else:
                graph_mult_networkx([mol1, mol2], ['OpenEye', 'RDKit'], toolkit_wrapper=rdkit_toolkit_wrapper, folder=folder, file_id=f"{i}")
        else: 
            if isinstance(mol1, Molecule):
                graph_networkx(mol1, folder=folder, toolkit_wrapper=rdkit_toolkit_wrapper, file_id=f"{i}")
            elif isinstance(mol2, Molecule):
                graph_networkx(mol2, folder=folder, toolkit_wrapper=rdkit_toolkit_wrapper, file_id=f"{i}")
            else:
                continue
    print(f"folder {folder} finished")

Folder is already there
Folder is already there
Folder is already there
Folder is already there
Folder is already there
Folder is already there
folder arom_kekule_diff finished
folder stereo_diff finished
folder proton_diff finished
folder failed_to_load finished
folder smiles_diff finished
folder inchi_diff finished


<Figure size 432x288 with 0 Axes>

# Summary of Results:
## arom_kekule_diff
These errors occur due to differences in interpreting individual bond orders around aromatic rings. The implications of this difference are small, so I chose to not investigate any further
## inchi_diff
Only one molecule, `DrugBank_6182', failed the InChI comparison. This is puzzling since the molecule passed all other tests, indicating that all atoms, bond orders, and connectivity are identical between the openeye and rdkit representation.
## proton_diff
In this case, the OpenEye representation of the molecule has one more proton than the RDKit molecule or original file. 
## smiles_diff
By close inspection, these differences occur when RDKit and OpenEye read the stereochemistry of pyramidal nitrogens differently. This shows up in the final smiles comparison and is not caught by the Molecule.are_isomorphic() function because the argument `strip_pyrimidal_n_atom_stereo` is set to true by default. Given that this is a known difference between the two toolkits, I decided to not investigate this any further. 
## stereo_diff
All of these errors occur around pyramidal sulfoxides. RDKit and OpenEye are assigning different stereochemistry to these sulfoxide stereocenters (ie. when one assings S the other assigns R). 
## failed_to_load
These errors occur when nitrogens have 4 bonds without the corresponding formal charge of +1. OpenEye seems account for this ambiguity by adding one proton to the nitrogen, while RDKit fails to load in the molecule. I have not yet found a way to coerce RDKit into loading in these molecules. 


# A look into each of these errors
## Arom_kekule_diff
These errors are not very important and are handled by the internal logic of `Molecule.are_isomorphic()`. I have included some examples of small molecules below (larger molecules do are not scaled quite so well when graphing in networkX). On the left is the networkX graphs of each molecule. Red edges indicates double bonds. On the right is the direct pymol representation of the molecule.

<h2><center>Drug_Bank_3674</center></h2>
<table><tr>
<td> <img src="netx_figs/arom_kekule_diff/DrugBank_3674_DrugBank_3674_(117).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/arom_kekule_diff/Drug_Bank_3674.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

<h2><center>Drug_Bank_4047</center></h2>
<table><tr>
<td> <img src="netx_figs/arom_kekule_diff/DrugBank_4047_DrugBank_4047_(162).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/arom_kekule_diff/Drug_Bank_4047.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

<h2><center>Drug_Bank_4247</center></h2>
<table><tr>
<td> <img src="netx_figs/arom_kekule_diff/DrugBank_4247_DrugBank_4247_(197).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/arom_kekule_diff/Drug_Bank_4247.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

<h2><center>Drug_Bank_6145</center></h2>
<table><tr>
<td> <img src="netx_figs/arom_kekule_diff/DrugBank_6145_DrugBank_6145_(122).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/arom_kekule_diff/Drug_Bank_6145.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

In all of these cases, RDKit appears to adjust the kekule structures, while OpenEye says the most "faithful" to the original structure. However, like I said before, this is a pretty minor issue and not worth spending any more time on. 


## inchi_diff
It is important to note that this error is only the result of differences in the final inchi string and not in the connectivity, stereochem (except possibly N), protons, etc. Printing the inchi strings of OFF_from_OE and OFF_from_RD gives the following code/output:

In [11]:
inchi_diff_mols = combined[combined['inchi_diff'] == True]
inchi_diff_mols.reset_index(inplace=True)
assert(len(inchi_diff_mols) == 1) # the following code relies on the fact that there is only one error of this type (change if working with a different data set)
inchi_from_oe = inchi_diff_mols.loc[0, 'OFF_from_RD']
inchi_from_rd = inchi_diff_mols.at[0, 'OFF_from_RD']
try:
    print(f"oe inchi using oe: {inchi_from_oe.to_inchi(toolkit_registry=open_eye_toolkit_wrapper)}")
    print(f"rd inchi using oe: {inchi_from_rd.to_inchi(toolkit_registry=open_eye_toolkit_wrapper)}")
    print(f"oe inchi using rd: {inchi_from_oe.to_inchi(toolkit_registry=rdkit_toolkit_wrapper)}")
    print(f"rd inchi using rd: {inchi_from_rd.to_inchi(toolkit_registry=rdkit_toolkit_wrapper)}")
except Exception as e:
    print(f"some kind of critical exception while converting to inchi: \n {e}")
print(f"molecule name: {inchi_from_oe.name}")

some kind of critical exception while converting to inchi: 
 Programming error: OpenEye atom stereochemistry assumptions failed.
molecule name: DrugBank_6182


Looking at the comparison table for this molecule gives the following row: 
126,DrugBank_6182,0,-999,3,-999,-99,-99,-99
This molecule seems to fail when working with openeye but is allowed by rdkit. The actual molecule gives the networkx graphing method a bit of trouble and looks like: 

<h2><center>Drug_Bank_6182</center></h2>
<table><tr>
<td> <img src="netx_figs/inchi_diff/DrugBank_6182_DrugBank_6182_(126).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/inchi_diff/DrugBank_6182.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

Looking at the molecule, I can see why openeye would have some trouble with the stereochemistry. Since this adamantane-like structure seems to be pretty rare and exotic, I will shelve this as a "problem-to-be-solved-later-if-it-ever-happens-again". 



## proton_diff
These errors are mostly due to ambiguous sdf info blocks, specifically incomplete or nonsensical information about formal charge. RDKit does nothing to these molecules, while openeye adds a proton during the Molecule creation step. Here are a few indicative test cases. 
<h2><center>DrugBank_1564</center></h2>
<table><tr>
<td> <img src="netx_figs/proton_diff/DrugBank_1564_DrugBank_1564_(184).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/proton_diff/DrugBank_1564.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>
<h2><center>DrugBank_1700</center></h2>
<table><tr>
<td> <img src="netx_figs/proton_diff/DrugBank_1700_DrugBank_1700_(211).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/proton_diff/DrugBank_1700.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>
<h2><center>DrugBank_2148</center></h2>
<table><tr>
<td> <img src="netx_figs/proton_diff/DrugBank_2148_DrugBank_2148_(297).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/proton_diff/DrugBank_2148.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>
<h2><center>DrugBank_2210</center></h2>
<table><tr>
<td> <img src="netx_figs/proton_diff/DrugBank_2210_DrugBank_2210_(305).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/proton_diff/DrugBank_2210.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>
<h2><center>DrugBank_2642</center></h2>
<table><tr>
<td> <img src="netx_figs/proton_diff/DrugBank_2642_DrugBank_2642_(356).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/proton_diff/DrugBank_2642.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>
<h2><center>DrugBank_5418</center></h2>
<table><tr>
<td> <img src="netx_figs/proton_diff/DrugBank_5418_DrugBank_5418_(11).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/proton_diff/DrugBank_5418.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

Let's look at the representation of DrugBank_2148. The following code grabs the sdf code block from the input file: 

In [12]:
name = 'DrugBank_2148'

for n, block in CustomSDFMolSupplier(drug_bank_file):
    if n == name:
        print(block)
p_diff_mols = combined[combined['name'] == name]
p_diff_mols.reset_index(inplace=True)
p_from_oe = inchi_diff_mols.loc[0, 'OFF_from_RD']
p_from_rd = inchi_diff_mols.at[0, 'OFF_from_RD']


DrugBank_2148
 OpenBabel01151909533D

 12 11  0  0  0  0  0  0  0  0999 V2000
   -0.5667   -1.9546   -0.0198 O   0  0  0  0  0  0  0  0  0  0  0  0
    1.7077    0.7192   -1.6950 O   0  0  0  0  0  0  0  0  0  0  0  0
   -2.4667   -0.1621    0.3151 O   0  0  0  0  0  0  0  0  0  0  0  0
   -1.2422   -0.2486   -1.9059 O   0  0  0  0  0  0  0  0  0  0  0  0
    2.2060   -0.2595    0.6064 O   0  0  0  0  0  0  0  0  0  0  0  0
   -0.0572    0.6171    0.1559 O   0  0  0  0  0  0  0  0  0  0  0  0
   -1.0428   -0.5672   -0.3338 P   0  0  0  0  0  0  0  0  0  0  0  0
    1.4947    0.9606   -0.2165 P   0  0  2  0  0  0  0  0  0  0  0  0
    2.0296    2.6803    0.5553 S   0  0  0  0  0  0  0  0  0  0  0  0
   -3.2615   -0.6876    0.0820 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4703   -0.3216   -2.5067 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.8972   -1.1756    0.4403 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  7  2  0  0  0  0
  3  7  1  0  0  0  0
  3 10  1  0  0  0  0
  4  7  1  0  0 

SDfile atom block columns have the following format.

`xxxxx.xxxxyyyyy.yyyyzzzzz.zzzzaaaddcccssshhhbbbvvvHHHrrriiimmmnnneee`

Following this specification, the sulfur in DrugBank_2148 has no implicit hydrogens and a formal charge of zero. This is nonsensical since this situation must result in a formal charge of -1. By quickly looking at the RDKit molecule representations, RDKit does not appear to try and rememdy this, since it did not add any hydrogens or any other formal charges (I checked this last point by inspecting each atoms in the RDKit molecule). 

Overall, this error appears to occur mostly around S and P's with impossible formal charge or implicit hydrogen properties. This may (and often does) cause openeye or rdkit warning messages and failures down the line, such as when trying to output the molecule to smiles. Overall, more testing is needed to identify all possible cases of this error and possible fixes. However, one possible fix could be refusing to load in molecules with impossible formal charges, such as the above example. 

# Stereochemistry Differences around Sulfoxides

The following error is made a bit more quantitative when looking at the given failure results. More testing needs to be done to find if this is an inevitable case for sulfoxides or happens only a fraction of the time. The following code tests the failure cases already found. 

In [13]:
dframe = combined

dframe = dframe[dframe['stereo_diff'] == True]
# define the dictinary that will store all stereo info
temp = {'name': None,
        'oe_N': [],
        'rd_N': [],
        'oe_C': [],
        'rd_C': [],
        'oe_S': [],
        'rd_S': [],
        'oe_etc': [],
        'rd_etc': []}
stereo_dframe = pd.DataFrame(columns=temp.keys())
# now check for stereochemistry for each molecule, atom-by-atom. Any stereocenters that are not
# N, C, or S will fall into the "etc" category and stored just in case
for index, row in dframe.iterrows():
    d = None
    d = deepcopy(temp)
    d['name'] = row['name']
    oe_mol = row['OFF_from_OE']
    rd_mol = row['OFF_from_RD']
    # iter through openeye molecule
    test = 1
    for atom in oe_mol.atoms:
        stereo = atom.stereochemistry
        if bool(stereo):
            el = atom.element.symbol
            if el == 'N':
                d['oe_N'].append(stereo)
            elif el == 'C':
                d['oe_C'].append(stereo)
            elif el == 'S':
                d['oe_S'].append(stereo)
            else:  # this step catches all other cases 
                d['oe_etc'].append((el, stereo))
    # iter through rdkit molecule
    for atom in rd_mol.atoms:
        stereo = atom.stereochemistry
        if bool(stereo):
            el = atom.element.symbol
            if el == 'N':
                d['rd_N'].append(stereo)
            elif el == 'C':
                d['rd_C'].append(stereo)
            elif el == 'S':
                d['rd_S'].append(stereo)
            else:  # this step catches all other cases 
                d['rd_etc'].append((el, stereo))
    # sort everything for readability 
    for key in d.keys():
        if isinstance(d[key], list):
            d[key].sort()

    stereo_dframe = stereo_dframe.append(d, ignore_index=True)
    print(TestMolecule.are_isomorphic(oe_mol, rd_mol))

big_df = pd.concat([dframe.reset_index(), stereo_dframe], axis = 1)
print(big_df[['index', 'name', 'oe_S', 'rd_S']])
big_df.to_csv("stereo_differences.csv")

(False, None)
(False, None)
(False, None)
(False, None)
(False, None)
(False, None)
(False, None)
   index           name           name oe_S rd_S
0    129  DrugBank_3817  DrugBank_3817  [R]  [S]
1    157  DrugBank_4032  DrugBank_4032  [S]  [R]
2    263  DrugBank_1971  DrugBank_1971  [R]  [S]
3    295  DrugBank_2140  DrugBank_2140  [R]  [S]
4    344  DrugBank_2563  DrugBank_2563  [R]  [S]
5    348  DrugBank_2585  DrugBank_2585  [R]  [S]
6    363  DrugBank_2687  DrugBank_2687  [S]  [R]


As seen above, printing the stereochemistry around sulfurs gives opposite results. As far as I know, this cannot be blamed on ambiguity in the SDfile, since (I think) stereochemistry is interpretted by the toolkit upon gathering info form the file. To determine which toolkit appear to be "right" be the human eye, I included the following 4 cases with the lone pair oriented behind the 3 pyramidal bonds. 

<h2>Drug_Bank_1971 (oe->R, rd->S, human->R)</h2> 
<img src="pymol_figs/stereo_diff/DrugBank_1971.png" alt="Drawing" style="width: 700px;"/>
<h2>Drug_Bank_2140 (oe->R, rd->S, human->R)</h2> 
<img src="pymol_figs/stereo_diff/DrugBank_2140.png" alt="Drawing" style="width: 700px;"/>
<h2>Drug_Bank_2563 (oe->R, rd->S, human->R)</h2> 
<img src="pymol_figs/stereo_diff/DrugBank_2563.png" alt="Drawing" style="width: 700px;"/>
<h2>Drug_Bank_4032 (oe->S, rd->R, human->S)</h2> 
<img src="pymol_figs/stereo_diff/DrugBank_4032.png" alt="Drawing" style="width: 700px;"/>


In conclusion, openeye agrees with what a human would assign to the stereochemistry, assuming the lone pair is the least significant group.


# failed_to_load
One general behavior of these errors is that openeye tends to create a molecule with one extra proton than what is originally in the sdf file. This can be shown using a similar test used before to test for protonization. Overall, this behavior is consistent with the previous protonization around sulfates and phosphates in that OpenEye will assume implicit hydrogens when the formal charge and hybridization require it. 

In [14]:
# first get all of the failed molecules
failed_molecules = combined[combined['failed_to_load'] == True]
names = failed_molecules['name2'].to_list()

failed_results = pd.DataFrame(columns=['name2', 'manual_reading', 'open_eye', 'proton_num(man vs oe)'])
for file_results, off_oe in zip(BlockToAtomSupplier(drug_bank_file), get_atom_nums(mols['OFF_from_OE'])):
    manual_name, manual_dict = file_results
    oe_name, oe_results = off_oe
    if (manual_name == oe_name) and (manual_name in names):
        row = pd.Series({'name2': manual_name, 
                'manual_reading': manual_dict, 
                'open_eye': oe_results,
                'proton_num(man vs oe)': f"{manual_dict.get('H')} vs {oe_results.get('H')}"})
        failed_results = failed_results.append(row, ignore_index=True)
        
print('all manual comparison differences for failed molecules:')
print(failed_results[['name2', 'proton_num(man vs oe)']])

all manual comparison differences for failed molecules:
            name2 proton_num(man vs oe)
0   DrugBank_2799              24 vs 26
1   DrugBank_5415              15 vs 17
2   DrugBank_3046                6 vs 7
3    DrugBank_472              21 vs 23
4    DrugBank_794              13 vs 15
5   DrugBank_3655                7 vs 9
6   DrugBank_3739              16 vs 18
7   DrugBank_6353              25 vs 26
8   DrugBank_1570                5 vs 7
9   DrugBank_1594               9 vs 11
10  DrugBank_1659              16 vs 17
11  DrugBank_1661              11 vs 12
12  DrugBank_4346              29 vs 31
13  DrugBank_1802              35 vs 37
14  DrugBank_6947              11 vs 13
15  DrugBank_7049              14 vs 16
16  DrugBank_4865              11 vs 13
17  DrugBank_2465                5 vs 7
18  DrugBank_2467              18 vs 19


Looking at some test cases gives the following results: 

<h2><center>DrugBank_2467</center></h2>
<table><tr>
<td> <img src="netx_figs/failed_to_load/DrugBank_2467(332).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/failed_to_load/DrugBank_2467.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

<h2><center>DrugBank_1570</center></h2>
<table><tr>
<td> <img src="netx_figs/failed_to_load/DrugBank_1570(188).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/failed_to_load/DrugBank_1570.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>

<h2><center>DrugBank_3046</center></h2>
<table><tr>
<td> <img src="netx_figs/failed_to_load/DrugBank_3046(35).png" alt="Drawing" style="width: 550px;"/> </td>
<td> <img src="pymol_figs/failed_to_load/DrugBank_3046.PNG" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>


In [15]:
name = 'DrugBank_3046'

for n, block in CustomSDFMolSupplier(drug_bank_file):
    if n == name:
        print(block)

DrugBank_3046
 OpenBabel01151909533D

 10  9  0  0  0  0  0  0  0  0999 V2000
    1.3029    1.3238   -1.7356 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7608   -0.6236    0.0866 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4196    0.2714    0.3990 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.9159    0.8629   -0.8007 N   0  0  0  0  0  0  0  0  0  0  0  0
    1.6735    1.7652   -2.6311 H   0  0  0  0  0  0  0  0  0  0  0  0
   -1.5664   -0.0491   -0.3803 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4691   -1.4283   -0.5961 H   0  0  0  0  0  0  0  0  0  0  0  0
   -1.1501   -1.0871    0.9990 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.2238   -0.3052    0.8658 H   0  0  0  0  0  0  0  0  0  0  0  0
    0.1237    1.0683    1.0878 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  4  3  0  0  0  0
  1  5  1  0  0  0  0
  2  3  1  0  0  0  0
  2  6  1  0  0  0  0
  2  7  1  0  0  0  0
  2  8  1  0  0  0  0
  3  4  1  0  0  0  0
  3  9  1  0  0  0  0
  3 10  1  0  0  0  0
M  END



# Brief Conclusion

Given the ambiguities in the SDFile format surrounding what constitutes an implicit hydrogen it is hard to prefer one toolkit over the other without more information. So far, OpenEye appears to handle these issues in the most intuitive way given the input SDF block, and this applies to sulfoxide stereochemistry as well. Fixes for the future will make an attempt at remedying the differences between OpenEye with minimal deviation from the original behavior and providing low-cost error checking for the user. This could be implimented by streamlining some of the methods in this notebook. 